In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import ttest_ind, ttest_ind_from_stats

In [ ]:
import datetime
from datetime import datetime,timedelta
from itertools import chain

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# Crate an engine
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# Get column names for measurement table
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
# Get column names for station table
columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

# Exploratory Climate Analysis

In [ ]:
# Retrive the last date point
last_date = (engine.execute('select date from measurement order by date desc').first())[0]
last_date

In [ ]:
# Calculate the date 1 year ago from the last data point in the database
year, month, day = map(int, last_date.split("-"))
year_ago = datetime(year, month, day) - timedelta(days=365)
print(year_ago.strftime("%Y-%m-%d"))

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
last_year_prcp = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= year_ago).all()
last_year_prcp

In [ ]:
# Perform a query to retrieve the data and precipitation scores
date = [row[0] for row in last_year_prcp]
precipitation =  [row[1] for row in last_year_prcp]

# Save the query results as a Pandas DataFrame and set the index to the date column
climate_df = pd.DataFrame({'date' : date,
                           'precipitation' : precipitation}).set_index('date')

# Sort the dataframe by date
climate_df = climate_df.sort_values('date')
climate_df.head()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
climate_df.plot(figsize = (10,6))
plt.xlabel("date")
plt.tick_params(
    axis='x',          
    which='both',      # both major and minor ticks are affected
    labelbottom=False) # remove x ticks label
plt.legend(loc = 'upper center')
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
climate_df.describe()

In [ ]:
# Design a query to show how many stations are available in this dataset?
session.query(func.count(Station.name)).all()

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
engine.execute('select station, count(station) as count from measurement group by station order by count desc').fetchall()

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
engine.execute('select min(tobs), max(tobs), avg(tobs) from measurement where station = "USC00519281"').fetchall()

In [ ]:
# Choose the station with the highest number of temperature observations.
engine.execute('select station, tobs from measurement where station = "USC00519281"').fetchall() 

In [ ]:
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
data = engine.execute('select tobs from Measurement where date >= "2016-08-23" and station = "USC00519281"').fetchall()
data = [row[0] for row in data]
hist_data = pd.DataFrame({'tobs': data})

In [ ]:
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
hist = hist_data.hist(bins = 12, figsize = (10,5))
plt.ylabel("Frequency")
plt.title("")
plt.legend(["tobs"])

# Optional Challenge Assignment

## Temperature Analysis I

In [ ]:
# Identify the average temperature in June at all stations across all available years in the dataset.
june = '06'
june_temperature = session.query(Measurement.tobs).filter(func.strftime("%m", Measurement.date) == june).all()

# Do the same for December temperature.
dec = '12'
dec_temperature = session.query(Measurement.tobs).filter(func.strftime("%m", Measurement.date) == dec).all()

# Use the t-test to determine whether the difference in the means
stats.ttest_ind(june_temperature, dec_temperature, equal_var = False)

## Temperature Analysis II

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
# trip date is 2018-01-16, 2018-01-22

my_trip = (calc_temps('2017-01-16', '2017-01-22'))
print(my_trip)

In [ ]:
# Create a dataframe using the result
my_trip_df = pd.DataFrame(my_trip, columns = ['min', 'avg', 'max'])
my_trip_df

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title, average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
error = [my_trip_df['max']-my_trip_df['min']]
my_trip_df.plot(kind = 'bar', y='avg', yerr=error, title='Trip Avg Temp', color='coral', alpha=0.5, figsize=(4,6), legend = '')
plt.ylabel("Temp (F)")
plt.tick_params(
    axis='x',          
    which='both', 
    labelbottom=False) 

## Daily Rainfall Average

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
engine.execute('select measurement.station, name, latitude, longitude, elevation, sum(prcp) as total_rainfall \
                from measurement\
                join station on measurement.station = station.station \
                where date between "2017-01-16" and "2017-01-22" \
                group by measurement.station order by total_rainfall desc').fetchall()

In [ ]:
session.query(Measurement.station, Station.name, Station.latitude, Station.longitude, Station.elevation,func.sum(Measurement.prcp)).\
                filter(Measurement.station == Station.station).filter(Measurement.date >= '2017-01-16').filter(Measurement.date <= '2017-01-22').\
                                                                                                            group_by(Measurement.station).all()

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# Set the start and end date of the trip
start_date = '2018-01-16'
end_date = '2018-01-22'

# Use the start and end date to create a range of dates
my_dates  = pd.date_range(start_date, end_date).strftime('%Y-%m-%d')
my_dates

In [ ]:
# Stip off the year and save a list of %m-%d strings
months_dates = pd.date_range(start_date, end_date).strftime('%m-%d')
months_dates

In [ ]:
normals = []
# Loop through the list of %m-%d strings and calculate the normals for each date
for date in months_dates:
    normal = daily_normals(date)
    # push each tuple of calculations into a list called `normals`
    normals.append(normal)
normals

In [ ]:
# Load the previous query results into a Pandas DataFrams
new_list = [x for x in chain.from_iterable(normals)]
my_trip_df = pd.DataFrame(new_list, columns = ['tmin','tavg','tmax'])

# Add the `trip_dates` range as the `date` index
my_trip_df['date'] = my_dates
my_trip_df = my_trip_df.set_index('date')
my_trip_df

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
my_trip_df.plot(kind = 'area', stacked = False, alpha = 0.25)
plt.xticks(rotation = 45)